In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
import pandas as pd
import json
import time
import numpy as np

In [2]:
client_id = "125fcec321e048db972933f9f364d74e"
client_secret = "615237e26e0c4d22a13889fd7b91a46a"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

## Get Basic Info about Artist

In [3]:
artist_name='Jessi'
artist_id1='64k5e9kV9MdukXjFrR5R37'

In [4]:
# in case we need multiple artists' data
artist_ids = [
 artist_id1 
]

In [5]:
basic_spotify_records = {
    'artist_name': [],
    'artist_genre': [],
    'followers': [],
    'popularity': []
}

for artist_id in artist_ids:
    result = sp.artist(artist_id)  # Search for the artist

    if result:
        artist_name = result['name']
        basic_spotify_records['artist_name'].append(artist_name)
        
        artist_genre = result['genres']
        basic_spotify_records['artist_genre'].append(artist_genre)
        
        artist_popularity = result['popularity']
        basic_spotify_records['popularity'].append(artist_popularity)
        
        artist_info = sp.artist(artist_id)
        followers = artist_info['followers']['total']
        basic_spotify_records['followers'].append(followers) 
               
    else:
        print(f"No artist found for {artist_id}")

# print(basic_spotify_records)

In [6]:
df_basic_spotify_records = pd.DataFrame(basic_spotify_records)
print(df_basic_spotify_records)

  artist_name artist_genre  followers  popularity
0       Jessi      [k-pop]    2145108          59


## Get Top 10 Tracks' Data

In [7]:
def get_top_songs_with_listeners(artist_id):
    try:
        # Get top tracks for the artist
        top_tracks = sp.artist_top_tracks(artist_id)
        # Initialize list to store top tracks data
        top_tracks_data = []

        # Iterate over top tracks
        for track_info in top_tracks['tracks']:
            # Extract relevant track information
            track_name = track_info['name']
            track_date = track_info['album']['release_date']
            track_popularity = track_info['popularity']

            # Append track data to list
            top_tracks_data.append({
                'Track Name': track_name,
                'Release Date': track_date,
                'Popularity': track_popularity
            })

        # Convert list of dictionaries to DataFrame
        df_top_songs = pd.DataFrame(top_tracks_data)
        return df_top_songs
    except Exception as e:
        print(f"Error occurred: {e}")
        return None


In [8]:
df_top_songs = get_top_songs_with_listeners(artist_id)
print(df_top_songs)

                               Track Name Release Date  Popularity
0                                    ZOOM   2022-04-13          71
1                               NUNU NANA   2020-07-30          67
2                                     Gum   2023-10-25          66
3                            Cold Blooded   2021-10-12          63
4           Shivers (feat. Jessi & SUNMI)   2021-10-28          59
5                          What Type of X   2021-03-17          56
6  Smoke (Prod. Dynamicduo, Padi) - Remix   2023-11-02          56
7                                My Romeo   2016-08-20          48
8                                   Gucci   2017-07-13          49
9                               Who Dat B   2020-07-30          46


## Getting More Songs Features

In [9]:
def get_artist_album_info(artist_ids):
    album_info = {
        'artist_name': [],
        'artist_id': [],
        'artist_genre': [],
        'album_id': [],
        'album_name': [],
        'album_type': [],
        'album_ttl_tracks': [],
        'release_date': [],
        'track_name': [],
        'track_duration_ms': [],
        'danceability': [],
        'energy': [],
        'key': [],
        'mode': [],
        'speechiness': [],
        'acousticness': [],
        'instrumentalness': [],
        'liveness': [],
        'valence': [],
        'tempo': [],
        'track_count_check': []
    }

    for artist_id in artist_ids:
        artist = sp.artist(artist_id)
        if artist:
            artist_name = artist['name']
            artist_genre = artist['genres']
            ttl_albums = [sp.artist_albums(artist_id, album_type=album_type) for album_type in ['album', 'single']]

            for album_set in ttl_albums:
                for album in album_set['items']:
                    album_id = album['id']
                    album_name = album['name']
                    album_type = album['album_type']
                    album_ttl_tracks = album['total_tracks']
                    release_date = album['release_date']

                    tracks = sp.album_tracks(album_id)
                    items = tracks['items']
                    track_count_check = 0
                    for track in items:
                        track_name = track['name']
                        track_duration_ms = track['duration_ms']

                        track_ids = [track['id'] for track in items]
                        features_list = sp.audio_features(track_ids)

                        album_info['artist_name'].append(artist_name)
                        album_info['artist_id'].append(artist_id)
                        album_info['artist_genre'].append(artist_genre)
                        album_info['album_id'].append(album_id)
                        album_info['album_name'].append(album_name)
                        album_info['album_type'].append(album_type)
                        album_info['album_ttl_tracks'].append(album_ttl_tracks)
                        album_info['release_date'].append(release_date)

                        album_info['track_name'].append(track_name)
                        album_info['track_duration_ms'].append(track_duration_ms)

                    for features in features_list:
                        danceability = features['danceability']
                        energy = features['energy']
                        key = features['key']
                        mode = features['mode']
                        speechiness = features['speechiness']
                        acousticness = features['acousticness']
                        instrumentalness = features['instrumentalness']
                        liveness = features['liveness']
                        valence = features['valence']
                        tempo = features['tempo']
                        track_count_check += 1

                        album_info['danceability'].append(danceability)
                        album_info['energy'].append(energy)
                        album_info['key'].append(key)
                        album_info['mode'].append(mode)
                        album_info['speechiness'].append(speechiness)
                        album_info['acousticness'].append(acousticness)
                        album_info['instrumentalness'].append(instrumentalness)
                        album_info['liveness'].append(liveness)
                        album_info['valence'].append(valence)
                        album_info['tempo'].append(tempo)
                        album_info['track_count_check'].append(track_count_check)

        else:
            print(f"Can not find artist {artist_name} with ID {artist_id} on Spotify")

    return album_info

In [10]:
album_data = get_artist_album_info(artist_ids)
album_data_df = pd.DataFrame(album_data)
print(album_data_df)

   artist_name               artist_id artist_genre                album_id  \
0        Jessi  64k5e9kV9MdukXjFrR5R37      [k-pop]  6bTs7ViUY2SgpBJiGqtceB   
1        Jessi  64k5e9kV9MdukXjFrR5R37      [k-pop]  6bTs7ViUY2SgpBJiGqtceB   
2        Jessi  64k5e9kV9MdukXjFrR5R37      [k-pop]  6bTs7ViUY2SgpBJiGqtceB   
3        Jessi  64k5e9kV9MdukXjFrR5R37      [k-pop]  6bTs7ViUY2SgpBJiGqtceB   
4        Jessi  64k5e9kV9MdukXjFrR5R37      [k-pop]  6bTs7ViUY2SgpBJiGqtceB   
5        Jessi  64k5e9kV9MdukXjFrR5R37      [k-pop]  6bTs7ViUY2SgpBJiGqtceB   
6        Jessi  64k5e9kV9MdukXjFrR5R37      [k-pop]  6bTs7ViUY2SgpBJiGqtceB   
7        Jessi  64k5e9kV9MdukXjFrR5R37      [k-pop]  6bTs7ViUY2SgpBJiGqtceB   
8        Jessi  64k5e9kV9MdukXjFrR5R37      [k-pop]  3y6bGlIVIIOQbOgQDRSw1T   
9        Jessi  64k5e9kV9MdukXjFrR5R37      [k-pop]  01pEJF6RE1w68zRXPwdTto   
10       Jessi  64k5e9kV9MdukXjFrR5R37      [k-pop]  01pEJF6RE1w68zRXPwdTto   
11       Jessi  64k5e9kV9MdukXjFrR5R37      [k-pop] 

## Get Related Artist Based On Spotify Recommendations Algorithms

In [19]:
def get_related_artists(artist_id):
    try:
        # Get related artists
        related_artists = sp.artist_related_artists(artist_id)
        # Extract related artists' names
        related_artist_names = [related_artist['name'] for related_artist in related_artists['artists']]
        df_related_artists=pd.DataFrame(related_artist_names)
        return df_related_artists
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

In [20]:
df_related_artists=get_related_artists(artist_id)
print(df_related_artists)

                              0
0                         HyunA
1                            CL
2                         Solar
3                    HyunA&DAWN
4                     JEON SOMI
5                         HWASA
6                          KARD
7                        DAWN 던
8                         SUNMI
9                      EVERGLOW
10                    Moon Byul
11                        AleXa
12                     CHUNG HA
13  Red Velvet - IRENE & SEULGI
14                  JEON SOYEON
15                          CLC
16                         ZICO
17                      Whee In
18                      Weeekly
19                     Jay Park


## Export to Excel

In [21]:
# Create a Pandas Excel writer
with pd.ExcelWriter(f'{artist_name}_spotify_data.xlsx', engine='xlsxwriter') as writer:
    # Write each DataFrame to a different sheet in the Excel file
    df_basic_spotify_records.to_excel(writer, sheet_name=f'Basic Info', index=False)
    df_top_songs.to_excel(writer, sheet_name=f'Top Tracks', index=False)
    album_data_df.to_excel(writer, sheet_name=f'Album Data', index=False)
    df_related_artists.to_excel(writer, sheet_name=f'Similar Artist', index=False)